In [ ]:
import os
import re

import pathlib
from pathlib import Path

import sqlite3

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
mpl.rcParams['figure.figsize'] = (16, 9)

In [ ]:
ROOT_DIR = "RawData"

In [ ]:
sns.set_style("ticks")
sns.color_palette("colorblind")
sns.set_context("notebook", font_scale=1.5)

# 1. Upload Datasets

In [ ]:
def get_data(filename, directory=ROOT_DIR, force_download=False):
    if force_download or not os.path.exists(filename):
        data = pd.ExcelFile(os.path.join(directory, filename))
    return data

In [ ]:
BE_data = get_data('BE_RawData_VPython.xlsx')
FR_data = get_data('FR_RawData_VPython.xlsx')
EU_data = get_data('EU_RawData_VPython.xlsx')

In [ ]:
print("BE_data, sheet names = \n {}\n".format(BE_data.sheet_names))
print("FR_data, sheet names = \n {}\n".format(FR_data.sheet_names))
print("EU_data, sheet names = \n {}\n".format(EU_data.sheet_names))

# 2. Postprocessing of the French and Belgian datasets

## a. Production of flat glass

In [ ]:
#Creating a DataFrame only with production data
FR_Prod = FR_data.parse('prod').set_index('year')
BE_Prod = BE_data.parse('prod').set_index('year')

#Creating a new DataFrame for Belgian and French flat glass production
BE_FR_FlatGlassProd = pd.concat([FR_Prod['flat glass, kt'], 
                                 BE_Prod['flat glass, kt']],
                                axis=1,
                                keys=['FR [kt]', 'BE [kt]'])

BE_FR_FlatGlassProd

In [ ]:
#Plotting FR and BE flat glass production
fig, ax = plt.subplots()

sns.lineplot(data=BE_FR_FlatGlassProd, ax=ax, markers=True, palette="GnBu")

ax.set_xlim(1945,2015)
ax.set_ylim(ymin=0)
ax.set_ylabel('Kilotons [kt]')

fig.suptitle('Flat glass production [kt/y]', fontsize = 20)
sns.despine()
plt.show()

In [ ]:
BE_FR_FlatGlassProd_roll = (
    BE_FR_FlatGlassProd.interpolate(method='pchip', limit_area='inside')
)

In [ ]:
# Linear interpolation for flat glass production in order to "roll"
BE_FR_FlatGlassProd_roll = BE_FR_FlatGlassProd.interpolate(method='pchip',
                                                           limit_area='inside')

# Rolling flat glass production according to a 5-year mean

BE_FR_FlatGlassProd_roll = (
    BE_FR_FlatGlassProd_roll.rolling(window=5, center=True).mean()
)

# Plotting rolled FR and BE flat glass production
fig, ax = plt.subplots()

sns.lineplot(data=BE_FR_FlatGlassProd_roll, ax=ax, palette="GnBu")

ax.set_xlim(1945, 2015)
ax.set_ylim(ymin=0)

fig.suptitle('Flat glass production (rolled) [kt/y]', fontsize=20)
sns.despine()
plt.show()

## b. Consumption of flat glass

In [ ]:
#Creating a DataFrame with import and export data
FR_Imp = FR_data.parse('import').set_index('year')
FR_Exp = FR_data.parse('export').set_index('year')

BE_Imp = BE_data.parse('import').set_index('year')
BE_Exp = BE_data.parse('export').set_index('year')

In [ ]:
BE_FR_Imp = pd.concat([FR_Imp['flat glass, kt'], BE_Imp['flat glass, kt']],
                      axis=1, keys=['FR [kt]', 'BE [kt]'])

BE_FR_Exp = pd.concat([BE_FR_Exp['flat glass, kt'], BE_FR_Exp['flat glass, kt']],
                      axis=1, keys=['FR [kt]', 'BE [kt]'])

In [ ]:
#Estimating the ratio importation/production and exportation/production
BE_FR_UseRatio = BE_FR_Exp.reset_index()[['year']].set_index('year')

BE_FR_UseRatio['FR, import ratio'] \
    = BE_FR_Imp['FR [kt]'] / BE_FR_FlatGlassProd['FR [kt]']
BE_FR_UseRatio['FR, export ratio'] \
    = BE_FR_Exp['FR [kt]'] / BE_FR_FlatGlassProd['FR [kt]']

BE_FR_UseRatio['BE, import ratio'] \
    = BE_FR_Imp['BE [kt]'] / BE_FR_FlatGlassProd['BE [kt]']
BE_FR_UseRatio['BE, export ratio'] \
    = BE_FR_Exp['BE [kt]'] / BE_FR_FlatGlassProd['BE [kt]']

BE_FR_UseRatio[-20:]

#Akima interpolation for missing data
BE_FR_UseRatio = BE_FR_UseRatio.interpolate(method='akima')

#Linear interpolation for the first years
BE_FR_UseRatio = BE_FR_UseRatio.interpolate(limit_direction='backward')

BE_FR_UseRatio[20:30]

### <span style='color: red;'>Proposition pour avoir **une** dataframe pratique et indexable</span>

In [ ]:
BE_FR = pd.concat([FR_Prod['flat glass, kt'], BE_Prod['flat glass, kt'],
                   FR_Imp['flat glass, kt'], BE_Imp['flat glass, kt'],
                   FR_Exp['flat glass, kt'], BE_Exp['flat glass, kt']],
                 axis=1, keys=[('Production [kt]', 'France'), ('Production [kt]', 'Belgium'),
                               ('Import [kt]', 'France'), ('Import [kt]', 'Belgium'),
                               ('Export [kt]', 'France'), ('Export [kt]', 'Belgium')],
                 names=['Info', 'Country']).stack()

BE_FR['Import Ratio'] = BE_FR['Import [kt]'] / BE_FR['Production [kt]']
BE_FR['Export Ratio'] = BE_FR['Export [kt]'] / BE_FR['Production [kt]']
BE_FR = BE_FR.unstack()

In [ ]:
# Interpolation for missing data
cols = [x for x in BE_FR.columns if x[0] in ['Import Ratio', 'Export Ratio']]

BR_FR[cols] = (
    BR_FR[cols]
        # Akima first
        .interpolate(method='akima')
        # Linear interpolation for the first years
        .interpolate(limit_direction='backward')
)

In [ ]:
BR_FR[cols].loc[1965:1974]

<p style='color: red; font-size: 18px; font-weight:bold;'>End of Proposition pour avoir **une** dataframe pratique et indexable</span>

In [ ]:
fig, ax = plt.subplots()

sns.lineplot(data=BE_FR_UseRatio, ax=ax, palette="GnBu")

ax.set_xlim(1945,2015)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

In [ ]:
BE_FR_Use = BE_FR_Exp.reset_index()[['year']].set_index('year')

BE_FR_Use['BE, flat glass consumption, [kt]'] = (
      BE_FR_FlatGlassProd['BE [kt]']
    + BE_FR_UseRatio['BE, import ratio']
    - BE_FR_UseRatio['BE, export ratio']
)

BE_FR_Use['FR, flat glass consumption, [kt]'] = (
      BE_FR_FlatGlassProd['FR [kt]']
    + BE_FR_UseRatio['FR, import ratio']
    - BE_FR_UseRatio['FR, export ratio']
)


BE_FR_Use[20:30]

In [ ]:
ToPlot = BE_FR_Use.reset_index()

fig, ax = plt.subplots()

sns.lineplot(data=ToPlot, x='year', y='FR, flat glass consumption, [kt]',
             ax=ax, palette="Reds")

ax.set_xlim(1945,2015)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

In [ ]:
sns.lineplot(data=ToPlot, x='year', 
             y='BE, flat glass consumption, [kt]',
             palette="Blues")
sns.despine()
plt.show()

In [ ]:
#Linear interpolation for flat glass consumption
BE_FR_Use_roll = BE_FR_Use.interpolate(method="linear", 
                                       limit_area = 'inside')

#Rolling flat glass production according to a 5-year mean
win_len = 5
BE_FR_Use_roll = BE_FR_Use_roll.rolling(win_len, 
                                   center = True).mean()

#Plotting rolled data
#sns.lineplot(data=BE_FR_Use_roll, palette="Blues")

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), sharex=True, sharey=True)

for i, (ax, col) in enumerate(zip(axes.flatten(), BE_FR_Use_roll.columns)):

    ax.plot(BE_FR_Use_roll.index, BE_FR_Use_roll[col])
    ax.set_title(col)
    ax.set_xlim(1945,2015)
    ax.set_ylim(0,1200)
    
sns.despine()
plt.show()

## c. Production of architectural flat glass

In [ ]:
#Share of building glass in the French production of flat glass
BldgShare = FR_Prod[['bldg glass/flat glass, %']]

ShareFR = BldgShare[['bldg glass/flat glass, %']].dropna()
print(ShareFR.isnull().sum())
ShareFR

In [ ]:
#Estimating parameters a and b of the regression lines

ShareFR['Trend1'] = ShareFR['bldg glass/flat glass, %'][:4]
ShareFR['Trend2'] = ShareFR['bldg glass/flat glass, %'][-5:]

ShareFR = ShareFR.reset_index()

x1 = list(ShareFR['year'][:4])
y1 = list(ShareFR['Trend1'][:4])
a1, b1 = np.polyfit(x1, y1, 1)

x2 = list(ShareFR['year'][-5:])
y2 = list(ShareFR['Trend2'][-5:])
a2, b2 = np.polyfit(x2, y2, 1)

#Point of intersection of the two regression lines
xi = (b1-b2) / (a2-a1)
print('Approximate year of rate change:', round(xi,0))

In [ ]:
fig, ax = plt.subplots()

sns.scatterplot(data = BldgShare, s=75, palette='Greys', ax=ax)

A = np.linspace(1945,xi)
ax.plot(A, a1 * A + b1, color='firebrick')
B = np.linspace(xi,2020)
ax.plot(B, a2 * B + b2, color='firebrick')

ax.set_ylim(0,1)
ax.set_xlim(1945,2015)
fig.suptitle('Share of building glass in the French flat glass production, %', 
             fontsize = 20)
sns.despine()
plt.show()

In [ ]:
#Update dataset according to regression lines
BldgShare = BldgShare.reset_index()

BldgShare.loc[(BldgShare['year'] < xi), 'bldg glass/flat glass, %'] \
    = round(a1 * BldgShare['year'] + b1, 2)

BldgShare.loc[(BldgShare['year'] >= xi), 'bldg glass/flat glass, %'] \
    = round(a2 * BldgShare['year'] + b2, 2)

BldgShare = BldgShare.set_index('year')
BldgShare

In [ ]:
#Estimate architectural flat glass production in France and Belgium
BE_FR_BldgGlassProd = BldgShare.reset_index()[['year']].set_index('year')

BE_FR_BldgGlassProd['FR [kt]'] \
    = (BE_FR_FlatGlassProd['FR [kt]']
       * BldgShare['bldg glass/flat glass, %'])

BE_FR_BldgGlassProd['BE [kt]'] \
    = (BE_FR_FlatGlassProd['BE [kt]']
       * BldgShare['bldg glass/flat glass, %'])

BE_FR_BldgGlassProd.head()

In [ ]:
#Plotting FR and BE bldg flat glass production
fig, ax = plt.subplots()

sns.lineplot(data=BE_FR_BldgGlassProd, markers=True, palette="GnBu", ax=ax)

ax.set_xlim(1945,2015)
ax.set_ylim(ymin=0)

fig.suptitle('Architectural flat glass production [kt/y]', fontsize = 20)
sns.despine()
plt.show()

In [ ]:
#Linear interpolation for bldg glass production in order to "roll"
BE_FR_BldgGlassProd_roll \
    = BE_FR_BldgGlassProd.interpolate(method = 'linear',
                                      limit_area = 'inside')

#Rolling flat glass production according to a 5-year mean
win_len = 5
BE_FR_BldgGlassProd_roll \
    = BE_FR_BldgGlassProd_roll.rolling(win_len,
                                       center = True).mean()

#Plotting results, to be compared with above graph
fig, ax = plt.subplots()

sns.lineplot(data=BE_FR_BldgGlassProd_roll, palette="GnBu", ax=ax)

ax.set_xlim(1945,2015)
ax.set_ylim(ymin=0)

fig.suptitle('Architectural flat glass production (rolled) [kt/y]', 
             fontsize = 20)
sns.despine()
plt.show()

## d. Production of insulating glass units in France

In [ ]:
#Creating a new DataFrame for French IGU
FR_IGU = FR_Prod[['IGU, "000 m²']]
FR_IGU = FR_IGU.rename(columns={'IGU, "000 m²':'prod ["000 m²]'})

FR_IGU['import ["000 m²]'] = FR_Imp['IGU, "000 m²']
FR_IGU['export ["000 m²]'] = FR_Exp['IGU, "000 m²']

FR_IGU['consumption ["000 m²]'] \
    = FR_IGU['prod ["000 m²]'] + FR_IGU['import ["000 m²]'] \
    - FR_IGU['export ["000 m²]']

FR_IGU[-10:]

In [ ]:
sns.scatterplot(data=FR_IGU, palette='colorblind')
sns.despine()
plt.show()

In [ ]:
FR_IGU['prod interpolated ["000 m²]'] = FR_IGU['prod ["000 m²]'].interpolate(method = 'pchip')

toplot = FR_IGU.reset_index()
sns.scatterplot(data=toplot, x='year', y='prod interpolated ["000 m²]')
sns.despine()
plt.show()

In [ ]:
#Estimating consumption/production ratio over the period 1995-2000
FR_IGU['consumption/production [-]'] \
    = FR_IGU['consumption ["000 m²]'] / FR_IGU['prod ["000 m²]']

a = FR_IGU.loc[1995:2000, 'consumption/production [-]'].mean().round(2)
print("Mean consumption/production ratio, 1995-2000:", a)

FR_IGU['consumption/production [-]'][1945] = 0
FR_IGU.head()

In [ ]:
#Estimating consumption since 1945
filt_null = FR_IGU['consumption ["000 m²]'].isnull()

FR_IGU.loc[filt_null, 'consumption ["000 m²]'] \
    = FR_IGU.loc[filt_null, 'prod interpolated ["000 m²]']

FR_IGU = FR_IGU.reset_index()

fig, ax = plt.subplots()

# lineplot is useless here. In fact you should use dataframe.plot() for both...
sns.lineplot(data=FR_IGU, x='year', y='prod interpolated ["000 m²]',
             ax=ax, palette='Reds')

ax.plot(FR_IGU['year'], FR_IGU['consumption ["000 m²]'], color='firebrick')

FR_IGU = FR_IGU.set_index('year')
sns.despine()
plt.show()

In [ ]:
#Rolling IGU consumption data according to a 5-year mean
FR_IGU_roll = FR_IGU[['consumption ["000 m²]']].copy()

win_len = 5
FR_IGU_roll = FR_IGU_roll.rolling(win_len, center = True).mean()

#Plotting results, to be compared with above graph
fig, ax = plt.subplots()

sns.lineplot(data = FR_IGU_roll, palette="GnBu")

ax.set_xlim(1945,2020)
ax.set_ylim(ymin=0)

fig.suptitle('IGU (rolled) [kt/y]', 
             fontsize = 20)

# 2. CO2 emissions of the French and Belgian flat glass industry since 1945

# a. Evolution of the carbon intensity

In [ ]:
#Creating a DataFrame only with CO2 emission data
Emissions = FR_data.parse('emissions').set_index('year')

CO2_Intensity = Emissions[['CO2 glass ind, kg/t']].copy()
CO2_Intensity.rename(columns={'CO2 glass ind, kg/t':'CO2 [kg/t]'}, 
           inplace=True)

CO2_Intensity.dropna()

In [ ]:
CO2_Intensity['CO2 interpolated [kg/t]'] \
    = CO2_Intensity['CO2 [kg/t]'].interpolate('pchip')

fig, ax = plt.subplots()
sns.lineplot(data=CO2_Intensity, palette="GnBu")

ax.scatter(x = CO2_Intensity.index, y = CO2_Intensity['CO2 [kg/t]'])

ax.set_xlim(1945,2020)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

## b. Absolute CO2 emissions since 1945

In [ ]:
CO2_ABS = CO2_Intensity.reset_index()[['year']].set_index('year')

CO2_ABS['FR flat glass [kt]'] \
    = BE_FR_FlatGlassProd['FR [kt]'] \
    * CO2_Intensity['CO2 interpolated [kg/t]'] / 1000
CO2_ABS['BE flat glass [kt]'] \
    = BE_FR_FlatGlassProd['BE [kt]'] \
    * CO2_Intensity['CO2 interpolated [kg/t]'] / 1000

CO2_ABS['FR bldg glass [kt]'] \
    = BE_FR_BldgGlassProd['FR [kt]'] \
    * CO2_Intensity['CO2 interpolated [kg/t]'] / 1000
CO2_ABS['BE bldg glass [kt]'] \
    = BE_FR_BldgGlassProd['BE [kt]'] \
    * CO2_Intensity['CO2 interpolated [kg/t]'] / 1000

CO2_ABS

In [ ]:
CO2_ABS['BE bldg glass [kt]'].dropna()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 16))

for i, (ax, col) in enumerate(zip(axes.flatten(), CO2_ABS.columns)):

    ax.plot(CO2_ABS.index, CO2_ABS[col], marker=".")
    ax.set_title(col)
    ax.set_xlim(1945,2015)
    ax.set_ylim(0,1400)
    
    # Annotate the outlier
    if 'BE' in col:
        x = 1950
        y = CO2_ABS.loc[1950, col]
        ax.scatter(x=x, y=y, c='r', marker='o', s=30)
        ax.annotate("Outlier", xy=(x,y), xycoords='data', color='r',
                    xytext=(20,40), textcoords='offset points',
                    arrowprops=dict(arrowstyle='simple', facecolor='r'))
    
sns.despine()
plt.show()

In [ ]:
#Linear interpolation for CO2 absolute emissions in order to "roll"
CO2_ABS_roll = CO2_ABS
CO2_ABS_roll = CO2_ABS_roll.interpolate(method='linear', 
                                        limit_area = 'inside')

#Rolling CO2 emissions according to a 5-year mean
win_len = 5
CO2_ABS_roll = CO2_ABS_roll.rolling(win_len,
                                    center = True).mean()

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 16), sharex=True, sharey=True)

for i, (ax, col) in enumerate(zip(axes.flatten(), CO2_ABS_roll.columns)):

    ax.plot(CO2_ABS_roll.index, CO2_ABS_roll[col])
    ax.set_title(col)
    ax.set_xlim(1945,2015)
    ax.set_ylim(0,1400)
    
sns.despine()
plt.show()

In [ ]:
#Base 100 = 1990
FlatGlassProd_diff \
    = 100 * BE_FR_FlatGlassProd_roll / BE_FR_FlatGlassProd_roll.loc[1980]

CO2Intensity_diff = 100 * CO2_Intensity / CO2_Intensity.loc[1980]

CO2ABS_diff = 100 * CO2_ABS_roll / CO2_ABS_roll.loc[1980]

#Plotting production, carbon intensity, CO2 absolute emissions for France
fig, ax = plt.subplots()
sns.lineplot(data = FlatGlassProd_diff, 
             x = FlatGlassProd_diff.index, 
             y = 'FR [kt]',
             palette = "GnBu",
             ax=ax)

ax.plot(CO2Intensity_diff.index, 
        CO2Intensity_diff['CO2 interpolated [kg/t]'])
ax.plot(CO2ABS_diff.index, 
        CO2ABS_diff['FR flat glass [kt]'])

ax.plot([1980, 2015], [100, 100], '--')

ax.set_xlim(1980,2015)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

In [ ]:
#Base 100 = 1990
FlatGlassProd_diff_noroll \
    = 100 * BE_FR_FlatGlassProd / BE_FR_FlatGlassProd.loc[1980]

CO2Intensity_diff_noroll = 100 * CO2_Intensity / CO2_Intensity.loc[1980]

CO2ABS_diff_noroll = 100 * CO2_ABS / CO2_ABS.loc[1980]

#Plotting production, carbon intensity, CO2 absolute emissions for France
fig, ax = plt.subplots()
sns.lineplot(data = FlatGlassProd_diff_noroll, 
             x = FlatGlassProd_diff_noroll.index, 
             y = 'FR [kt]',
             palette = "GnBu")

ax.plot(CO2Intensity_diff_noroll.index, 
        CO2Intensity_diff_noroll['CO2 interpolated [kg/t]'])
ax.plot(CO2ABS_diff_noroll.index, 
        CO2ABS_diff_noroll['FR flat glass [kt]'])

ax.plot([1980, 2015], [100, 100], '--')

ax.set_xlim(1980,2015)
ax.set_ylim(ymin=0)

sns.despine()
plt.show()

## c. CO2 emissions relating to IGU consumption in FR

In [ ]:
#Hypothesis: The average thickness of an IGU is twice 4mm, 
#i.e., 8mm of glass, which equals 20kg per m²
FR_IGU['consumption [kt]'] = FR_IGU['consumption ["000 m²]'] * 20 / 1000

#Absolute CO2 emissions in France
CO2_IGU = FR_IGU['consumption [kt]'] \
    * CO2_Intensity['CO2 interpolated [kg/t]'] / 1000

CO2_IGU = pd.DataFrame(CO2_IGU, columns = ['IGU CO2 [kt]'])

#Plotting
sns.lineplot(data = CO2_IGU)

sns.despine()
plt.show()

In [ ]:
#Rolling CO2 emissions of IGU according to a 5-year mean
win_len = 5
CO2_IGU_roll = CO2_IGU.rolling(win_len, center = True).mean()

#Plotting
fig, ax = plt.subplots()

sns.lineplot(data = CO2_IGU_roll)

ax.set_xlim(1945,2020)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

In [ ]:
#Base 100 = 1990
CO2_IGU_diff_roll = 100 * CO2_IGU_roll / CO2_IGU_roll.loc[1980]
FR_IGU_diff_roll = 100 * FR_IGU_roll / FR_IGU_roll.loc[1980]

#Plotting consumption, carbon intensity, CO2 absolute emissions for France
fig, ax = plt.subplots()

sns.lineplot(data = FR_IGU_diff_roll, 
             x = FR_IGU_diff_roll.index, 
             y = 'consumption ["000 m²]',
             palette = "GnBu")

ax.plot(CO2Intensity_diff_noroll.index, 
        CO2Intensity_diff_noroll['CO2 interpolated [kg/t]'])
ax.plot(CO2_IGU_diff_roll.index, 
        CO2_IGU_diff_roll['IGU CO2 [kt]'])

ax.plot([1980, 2020], [100, 100], '--')

ax.set_xlim(1980,2020)
ax.set_ylim(ymin=0)
sns.despine()
plt.show()

# 3. Postprocessing of the European dataset

## a. Production of flat glass

In [ ]:
EU_Prod = EU_data.parse('prod').set_index(['Nbr of countries', 'year'])
EU_Prod

In [ ]:
#Plot the European production of flat glass

EUplot = EU_Prod.reset_index()

fig, ax = plt.subplots()

sns.lineplot(data = EUplot, x = 'year', y = 'flat glass, kt', 
             hue = 'Nbr of countries', 
             palette = 'colorblind',
             ax = ax)

ax.set_xlim(1945,2020)
ax.set_ylim(ymin=0)
fig.suptitle('European production of flat glass [kt/y]', fontsize = 20)
sns.despine()
plt.show()

In [ ]:
#Adding import and export in order to estimate consumption in EU
EU_Imp = EU_data.parse('import').set_index(['Nbr of countries', 'year'])
EU_Exp = EU_data.parse('export').set_index(['Nbr of countries', 'year'])

#Estimating consumption in EU
EU_Consumption = EU_Prod.reset_index()[['Nbr of countries', 
                                        'year']].set_index(['Nbr of countries', 
                                                            'year'])
EU_Consumption['flat glass, kt'] = EU_Prod['flat glass, kt'] \
    + EU_Imp['flat glass, kt'] - EU_Exp['flat glass, kt']

#Plot the European consumption of flat glass

EUplot = EU_Consumption.reset_index()

fig, ax = plt.subplots()

sns.lineplot(data = EUplot, x = 'year', y = 'flat glass, kt', 
             hue = 'Nbr of countries', 
             palette = 'colorblind',
             ax = ax)

ax.set_xlim(1945,2020)
ax.set_ylim(ymin=0)
fig.suptitle('European consumption of flat glass [kt/y]', fontsize = 20)
sns.despine()
plt.show()

## c. Production of architectural flat glass

In [ ]:
#Share of building glazing in the European production of flat glass
EUBldgShare = EU_Prod[['bldg glass/flat glass, %']]

ShareEU = EUBldgShare.dropna()
ShareEU

In [ ]:
toplot = EUBldgShare.reset_index()

fig, ax = plt.subplots()

sns.lineplot(data=toplot, x='year', y='bldg glass/flat glass, %', 
             markers="o", ax=ax)

ax.lines[0].set_linestyle("--")
ax.set_xlim(1945,2020)
ax.set_ylim(0,1)

In [ ]:
EUBldgShare

In [ ]:
#Estimating parameters a, b and c; 2-degree polynomial
values = np.polyfit(x=ShareEU.index.get_level_values(0), 
                  y=ShareEU['bldg glass/flat glass, %'], 
                  deg=2, full=True) 

values

In [ ]:
#Interpolation according to values calculated above
EUBldgShare.loc[1970:2020, 'bldg glass/flat glass (interpolated) [%]'] \
    = values[0][0] * EUBldgShare.index(0) ** 2\
    * values[0][1] * EUBldgShare.index(0) \
    * values[0][2] * EUBldgShare.index(0)

EUBldgShare[-50:]

In [ ]:
fig, ax = plt.subplots()

sns.scatterplot(data=EUBldgShare,
                x='year', 
                y='bldg glass/flat glass (interpolated) [%]',
                ax=ax)

ax.lines[0].set_linestyle("--")
ax.set_xlim(1945,2020)
ax.set_ylim(0,1)